<a href="https://colab.research.google.com/github/cr21/Amazon-Fine-Food-Reviews/blob/master/FeatureSelectionChi2Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
cd drive/MyDrive/Dataset/

[Errno 2] No such file or directory: 'drive/MyDrive/Dataset/'
/content/drive/MyDrive/Dataset


In [127]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection  import mutual_info_classif
from sklearn import linear_model

# Feature Selection using Chi2 Test

In [125]:
# Load 20 News Group dataset and split train and test data
def load20NG():
  dataset = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'), random_state= 12)
  data = dataset.data
  labels = dataset.target

  vectorizer = TfidfVectorizer(stop_words='english',min_df = 5)

  # # vectorize train data
  ngVectorData = vectorizer.fit_transform(data)
  featureNames = vectorizer.get_feature_names()
  return data, labels, ngVectorData, featureNames
data, labels, ng20Data, ng20featureNames = load20NG()
X_train, X_test, y_train, y_test = train_test_split(ng20Data, labels, test_size=0.33, random_state=42)

In [238]:
Transformed_200_X_train = SelectKBest(chi2, k=200).fit_transform(X_train, y_train)

In [239]:
Transformed_200_X_test = SelectKBest(chi2, k =200).fit_transform(X_test,y_test)

In [240]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(Transformed_200_X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [241]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_train, y_train)}")

 training Error : 0.5355615396800253


In [242]:
#testing error
print(f" testing Error : {ngClassfier.score(Transformed_200_X_test, y_test)}")

 testing Error : 0.09212218649517685


# Mutual Information for Feature Selection

In [243]:
feature_scores = mutual_info_classif(X_train, y_train, random_state=0)

In [244]:
Transformed_200_X_train = SelectKBest(mutual_info_classif, k=200).fit_transform(X_train, y_train)


In [245]:
Transformed_200_X_test = SelectKBest(mutual_info_classif, k =200).fit_transform(X_test,y_test)

In [246]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(Transformed_200_X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [247]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_train, y_train)}")

 training Error : 0.3446063678124505


In [248]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_test, y_test)}")

 training Error : 0.13167202572347267


In [249]:
# Top 200 features
sorted_ngFeatureList = np.argsort(feature_scores)
reverse_sorted_ngFeatureList = sorted_ngFeatureList[::-1]
top200 = reverse_sorted_ngFeatureList[:200]
for id, featureId in enumerate(top200):
  featureNameList = np.array(ng20featureNames)[featureId]
  print(f" id : {id} , feature name : {featureNameList}")


 id : 0 , feature name : like
 id : 1 , feature name : just
 id : 2 , feature name : don
 id : 3 , feature name : know
 id : 4 , feature name : think
 id : 5 , feature name : people
 id : 6 , feature name : does
 id : 7 , feature name : time
 id : 8 , feature name : use
 id : 9 , feature name : good
 id : 10 , feature name : way
 id : 11 , feature name : make
 id : 12 , feature name : new
 id : 13 , feature name : ve
 id : 14 , feature name : used
 id : 15 , feature name : want
 id : 16 , feature name : say
 id : 17 , feature name : right
 id : 18 , feature name : need
 id : 19 , feature name : did
 id : 20 , feature name : thanks
 id : 21 , feature name : really
 id : 22 , feature name : work
 id : 23 , feature name : problem
 id : 24 , feature name : edu
 id : 25 , feature name : using
 id : 26 , feature name : believe
 id : 27 , feature name : point
 id : 28 , feature name : sure
 id : 29 , feature name : things
 id : 30 , feature name : years
 id : 31 , feature name : help
 id : 32

In [250]:
top200XTrain = X_train[:,top200 ]
top200XTest = X_test[:, top200]

In [251]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(top200XTrain, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [252]:
# training error
print(f" training Error : {ngClassfier.score(top200XTrain, y_train)}")

 training Error : 0.3446063678124505


In [253]:
#testing error
print(f" testing Error : {ngClassfier.score(top200XTest, y_test)}")

 testing Error : 0.30884244372990355


# L1 feature selection on text

In [254]:
ngClassfierL1 = LogisticRegression(penalty='l1', tol =0.001, C = 1.0,n_jobs=-1, solver='saga',max_iter = 100)
ngClassfierL1.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l1',
                   random_state=None, solver='saga', tol=0.001, verbose=0,
                   warm_start=False)

In [255]:
clf = linear_model.Lasso(alpha=0.01,max_iter=2)
clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:460: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3323.7840081453323, tolerance: 39.247570893394574
  max_iter, tol, rng, random, positive)


Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=2, normalize=False,
      positive=False, precompute=False, random_state=None, selection='cyclic',
      tol=0.0001, warm_start=False)

In [256]:
sorted_ngFeatureList = np.argsort(clf.coef_)
reverse_sorted_ngFeatureList = sorted_ngFeatureList[::-1]
top200 = reverse_sorted_ngFeatureList[:200]
for id, featureId in enumerate(top200):
  featureNameList = np.array(ng20featureNames)[featureId]
  print(f" id : {id} , feature name : {featureNameList}")

 id : 0 , feature name : people
 id : 1 , feature name : israel
 id : 2 , feature name : government
 id : 3 , feature name : gun
 id : 4 , feature name : children
 id : 5 , feature name : fbi
 id : 6 , feature name : jews
 id : 7 , feature name : jesus
 id : 8 , feature name : god
 id : 9 , feature name : did
 id : 10 , feature name : space
 id : 11 , feature name : dy
 id : 12 , feature name : e3
 id : 13 , feature name : dx50
 id : 14 , feature name : e5
 id : 15 , feature name : dx2
 id : 16 , feature name : e6
 id : 17 , feature name : e7
 id : 18 , feature name : e8
 id : 19 , feature name : e9
 id : 20 , feature name : e_
 id : 21 , feature name : ea
 id : 22 , feature name : dx
 id : 23 , feature name : eachus
 id : 24 , feature name : eager
 id : 25 , feature name : eagerly
 id : 26 , feature name : eagle
 id : 27 , feature name : eagles
 id : 28 , feature name : ear
 id : 29 , feature name : e4
 id : 30 , feature name : e2
 id : 31 , feature name : dye
 id : 32 , feature name 

In [257]:
top200XTrain = X_train[:,top200 ]
top200XTest = X_test[:, top200]

In [258]:
ngClassfierL1 = LogisticRegression(penalty='l1', tol =0.001, C = 1.0,n_jobs=-1, solver='saga',max_iter = 100)
ngClassfierL1.fit(top200XTrain, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l1',
                   random_state=None, solver='saga', tol=0.001, verbose=0,
                   warm_start=False)

In [259]:
ngClassfierL1.score(top200XTrain, y_train)

0.1910343735149691

In [260]:
ngClassfierL1.score(top200XTest, y_test)

0.17572347266881028

In [261]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True,)

In [262]:
X_new_train = model.transform(X_train)
X_new_test = model.transform(X_test)

In [263]:
ngClassfierL1 = LogisticRegression(penalty='l1', tol =0.001, C = 1.0,n_jobs=-1, solver='saga',max_iter = 100)
ngClassfierL1.fit(X_new_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l1',
                   random_state=None, solver='saga', tol=0.001, verbose=0,
                   warm_start=False)

In [264]:
ngClassfierL1.score(X_new_train,y_train)

0.22144780611436718

In [265]:
ngClassfierL1.score(X_new_test,y_test)

0.22363344051446946